# Implementation of PLSI algorithm using Spark 

In [1]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import findspark
findspark.init("spark-2.2.3-bin-hadoop2.6")
import pyspark
from numpy import random


In [22]:
from pyspark import sql

## 1. Prepare the environment

### Import the data

We will implement the PLSI algorithm on the Movielens dataset. To simplify our task, we will start by implementing the PLSI algorithm on a reduced version of the Movielens dataset ("ratings_short"), which contains 100 836 observations. 

In [3]:
ratings_data = pd.read_csv("ratings.csv")
ratings_data.head()

,user_id,user_username,movie_id,rating
0,2,William,1768,1
1,3,James,615,3
2,7,Joseph,82,3
3,7,Joseph,532,3
4,8,Thomas,698,3


**Description of the dataset :** 
- userId, to characterize the users 
- movieId, to characterize the movies
- rating : the rating of the user to the corresponding movie. Ratings are going from 1 to 5 but here we will only use the seen / not seen information to provide movie recommendations.  
- timestamp : we will not be using this column

More information about the movies are available in the "movies.csv" dataset : the movieId gives us access to the corresponding movie information such as the title and the genres. 

In [4]:
movies = pd.read_csv("movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
####MORE DESCRIPTIONS ON THE DATA

### Create a Spark environment

In [5]:
sc = pyspark.SparkContext()

To be able to implement the PLSI algorithm in Spark, we will need to transform the dataset into an RDD, and then perform pyspark operations on it. 

In [55]:
rdd = sc.textFile("ratings.csv")

#Remove header line 
header = rdd.first()
rdd = rdd.filter(lambda x: x != header)

rdd.collect()[0:10]

['2,William,1768,1',
 '3,James,615,3',
 '7,Joseph,82,3',
 '7,Joseph,532,3',
 '8,Thomas,698,3',
 '10,Robert,1693,3',
 '11,Edward,615,1',
 '18,David,1,3',
 '18,David,28,3',
 '18,David,1596,5']

## 2. Probabilistic Latent Semantic Indexing algorithm (PLSI)

The PLSI algorithm that we will implement here is based on Das description of the Google News recommendation system. The algorithm is based on the following model : 
- u (users) and s (movies) are random variables 
- The relationship between users and movies is learned by modeling the joint distribution of users and items as a mixture distribution 
- To capture this relationship, we introduce a hidden variable z (latent variable), that kind of represents user communities (same preferences) and movie communities (sames genres). 

All in all, we try to compute the following probability for each (user, movie) couple : p(s|u) = sum(p(s|z)p(z|u)), which is the probability for a given user to see a given movie. This is obtained by summing for each community the probability for a movie s to be seen given a community z times the probability to be in the community z given a user u. 

### Going through the algorithm : main steps

**INITIALISATION**

**E-STEP - Compute q( z | (u,s) ) : the probability that the (user, movie) couple belongs to the class z**
This step is first initialized at random :
- To each couple (u,s), assign each possible community 
- Ex with number of classes = 2 : the lines (Marie, Star Wars) and (Gaëlle, Matrix) will give (Marie, Star Wars, 1), (Marie, Star Wars, 2), (Gaëlle, Matrix, 1), (Gaëlle, Matrix, 2)
- To each line, assign a random probability. This random probability corresponds to q*( z | (u,s) ). For example if I have (Marie, Star Wars, 1, 0.3), then the probability that the couple (Marie, Star Wars) is in class 1 is 0.3. 

LogLik = 0

**ITERATION**

**M-STEP - Compute p(s|z) and p(z|u) based on q( z | (u,s) )**
- Compute p(s | z) :  sum the probas associated to every couple (s,z) and divide it by the sum of probas associated to this z
- Compute p(z | u) : sum the probas associated to every couple (u,z) and divide by the sum of probas associated to this u

**E-STEP - Compute new q( z | (u,s) ) = p(s|z)p(z|u) / ∑p(s|z)p(z|u)**
- For each (u,s,z), compute p(s | z) * p(z | u)
- For each (u,s), compute ∑ p(s | z)* p(z | u) (summing over z)     ***(this corresponds to p(s|u))***
- For each (u,s,z), compute p(s|z)p(z|u) / ∑p(s|z)p(z|u)             ***(this corresponds to the new q( z | (u,s) )***
	    
**Update LogLik** = sum( log( ∑ p(s | z) * p(z | u))) = sum( log (p(s | u))

**Iterate again until LogLik converges** : this means that it has reached its maximum and we have found the best estimation of p(z | u) and p(s | z).
	    
**We can now predict the probability that Gaëlle will watch Star Wars** :
p(Star Wars | Gaëlle) = p( 1 | Gaëlle) * p(Star Wars |1) + p(2 | Gaëlle) * p(Star Wars | 2)

### Implementing the algorithm 

Keep only (user, movie) information : 

In [56]:
rdd = rdd.map(lambda line : line.split(',')).map(lambda line : line[0] + ',' + line[2])

In [57]:
rdd.collect()[0:10]

['2,1768',
 '3,615',
 '7,82',
 '7,532',
 '8,698',
 '10,1693',
 '11,615',
 '18,1',
 '18,28',
 '18,1596']

#### Initialisation of q : (first E-Step)

To each couple (u,s), assign each possible community z : 

In [58]:
nb_z = 3 #number of classes
classes = sc.parallelize(range(nb_z))
classes.collect()
rdd = rdd.cartesian(classes)

In [59]:
rdd.collect()[0:10]

[('2,1768', 0),
 ('3,615', 0),
 ('7,82', 0),
 ('7,532', 0),
 ('8,698', 0),
 ('10,1693', 0),
 ('11,615', 0),
 ('18,1', 0),
 ('18,28', 0),
 ('18,1596', 0)]

To each line, assign a random probability :

In [71]:
q = rdd.map(lambda x : (x, np.random.rand()))
q = q.map(lambda x : (x[0][0].split(','), x[0][1], x[1]))

In [75]:
q.collect()[0:10]

[(['2', '1768'], 0, 0.0022266067292384673),
 (['3', '615'], 0, 0.6952350116531781),
 (['7', '82'], 0, 0.7036829130103541),
 (['7', '532'], 0, 0.9535123375904958),
 (['8', '698'], 0, 0.05904050841113617),
 (['10', '1693'], 0, 0.4191670103214322),
 (['11', '615'], 0, 0.4316767734336888),
 (['18', '1'], 0, 0.9857750440664248),
 (['18', '28'], 0, 0.8333981157619267),
 (['18', '1596'], 0, 0.48291949455861816)]

#### One iteration step 

M-STEP - Compute p(s|z) and p(z|u) based on q( z | (u,s) )

Compute p(s | z) : sum the probas associated to every couple (s,z) and divide it by the sum of probas associated to this z

In [94]:
SZ_probas = q.map(lambda x: ((x[0][1], x[1]), x[2]))

In [122]:
Nsz = SZ_probas.reduceByKey(lambda x,y: x + y)

In [159]:
Z_probas = q.map(lambda x: (x[1], x[2]))

In [105]:
Nz = Z_probas.reduceByKey(lambda x,y: x+y)

In [123]:
Nsz = Nsz.map(lambda x : (x[0][1], (x[0][0], x[1])))

In [124]:
Psz = Nsz.join(Nz)

In [126]:
Psz = Psz.map(lambda x : ((x[1][0][0], x[0]), x[1][0][1] / x[1][1]))

Compute p(z | u) : sum the probas associated to every couple (u,z) and divide by the sum of probas associated to this u

In [128]:
ZU_probas = q.map(lambda x: ((x[0][0], x[1]), x[2]))

In [151]:
Nzu = ZU_probas.reduceByKey(lambda x,y: x + y)

In [140]:
U_probas = q.map(lambda x: (x[0][0], x[2]))

In [142]:
Nu = U_probas.reduceByKey(lambda x,y: x+y)

In [153]:
Nzu = Nzu.map(lambda x : (x[0][0], (x[0][1], x[1])))

In [155]:
Pzu = Nzu.join(Nu)

In [156]:
Pzu = Pzu.map(lambda x : ((x[1][0][0], x[0]), x[1][0][1] / x[1][1]))

E-STEP - Compute new q( z | (u,s) ) = p(s|z)p(z|u) / ∑p(s|z)p(z|u)

For each (u,s,z), compute p(s | z) * p(z | u)

In [161]:
Pzu.collect()[0:10]

[((1, '10'), 0.3333333333333333),
 ((0, '10'), 0.3333333333333333),
 ((2, '10'), 0.3333333333333333),
 ((1, '44'), 0.3333333333333333),
 ((0, '44'), 0.3333333333333333),
 ((2, '44'), 0.3333333333333333),
 ((1, '54'), 0.3333333333333333),
 ((0, '54'), 0.3333333333333333),
 ((2, '54'), 0.3333333333333333),
 ((1, '102'), 0.3333333333333333)]

In [162]:
Psz.collect()[0:10]

[(('1', 0), 0.005685184977099493),
 (('1596', 0), 0.012360966432698435),
 (('130', 0), 0.03684078110308122),
 (('68', 0), 0.03806300471276274),
 (('517', 0), 0.04728579674309403),
 (('598', 0), 0.02886330455948453),
 (('1856', 0), 0.0033575666175042276),
 (('564', 0), 0.027436949758408432),
 (('1293', 0), 0.006810781885253489),
 (('2767', 0), 0.005893079737151274)]

In [163]:
Pzu = Pzu.map(lambda x : (x[0][0], (x[0][1], x[1])))

In [164]:
Psz = Psz.map(lambda x : (x[0][1], (x[0][0], x[1])))

In [165]:
Pzu.collect()[0:10]

[(1, ('10', 0.3333333333333333)),
 (0, ('10', 0.3333333333333333)),
 (2, ('10', 0.3333333333333333)),
 (1, ('44', 0.3333333333333333)),
 (0, ('44', 0.3333333333333333)),
 (2, ('44', 0.3333333333333333)),
 (1, ('54', 0.3333333333333333)),
 (0, ('54', 0.3333333333333333)),
 (2, ('54', 0.3333333333333333)),
 (1, ('102', 0.3333333333333333))]

In [166]:
Psz.collect()[0:10]

[(0, ('1', 0.005685184977099493)),
 (0, ('1596', 0.012360966432698435)),
 (0, ('130', 0.03684078110308122)),
 (0, ('68', 0.03806300471276274)),
 (0, ('517', 0.04728579674309403)),
 (0, ('598', 0.02886330455948453)),
 (0, ('1856', 0.0033575666175042276)),
 (0, ('564', 0.027436949758408432)),
 (0, ('1293', 0.006810781885253489)),
 (0, ('2767', 0.005893079737151274))]

In [181]:
PzuPsz = Pzu.join(Psz).map(lambda x : ((x[1][0][0], x[1][1][0]), (x[0], x[1][0][1]*x[1][1][1])))

In [182]:
PzuPsz.collect()[0:10]

[(('10', '1'), (0, 0.0018950616590331642)),
 (('10', '1596'), (0, 0.0041203221442328115)),
 (('10', '130'), (0, 0.01228026036769374)),
 (('10', '68'), (0, 0.01268766823758758)),
 (('10', '517'), (0, 0.015761932247698007)),
 (('10', '598'), (0, 0.009621101519828177)),
 (('10', '1856'), (0, 0.0011191888725014091)),
 (('10', '564'), (0, 0.009145649919469477)),
 (('10', '1293'), (0, 0.0022702606284178296)),
 (('10', '2767'), (0, 0.0019643599123837577))]

For each (u,s), compute ∑ p(s | z)* p(z | u) (summing over z) (this corresponds to p(s|u))

In [183]:
SumPzuPsz = PzuPsz.map(lambda x : (x[0], x[1][1])).reduceByKey(lambda x,y : x+y)

In [184]:
SumPzuPsz.collect()[0:10]

[(('10', '770'), 0.041272879250374275),
 (('10', '1097'), 0.009934157939033121),
 (('44', '2498'), 0.011040179950712641),
 (('44', '1550'), 0.0036719624506974646),
 (('54', '770'), 0.041272879250374275),
 (('54', '1097'), 0.009934157939033121),
 (('102', '770'), 0.041272879250374275),
 (('102', '1097'), 0.009934157939033121),
 (('112', '2498'), 0.011040179950712641),
 (('112', '1550'), 0.0036719624506974646)]

For each (u,s,z), compute p(s|z)p(z|u) / ∑p(s|z)p(z|u) (this corresponds to the new q( z | (u,s) )

In [185]:
q1 = PzuPsz.join(SumPzuPsz)

In [187]:
q1.collect()[0:10]

[(('10', '770'), ((0, 0.01344994304525593), 0.041272879250374275)),
 (('10', '770'), ((1, 0.013911468102559173), 0.041272879250374275)),
 (('10', '770'), ((2, 0.013911468102559173), 0.041272879250374275)),
 (('54', '1097'), ((0, 0.0015842162183411067), 0.009934157939033121)),
 (('54', '1097'), ((1, 0.004174970860346007), 0.009934157939033121)),
 (('54', '1097'), ((2, 0.004174970860346007), 0.009934157939033121)),
 (('102', '770'), ((0, 0.01344994304525593), 0.041272879250374275)),
 (('102', '770'), ((1, 0.013911468102559173), 0.041272879250374275)),
 (('102', '770'), ((2, 0.013911468102559173), 0.041272879250374275)),
 (('190', '770'), ((0, 0.01344994304525593), 0.041272879250374275))]

In [188]:
q1 = q1.map(lambda x : (x[0], x[1][0][0], x[1][0][1]/x[1][1]))

In [193]:
q1.collect()[0:10]

[(('10', '770'), 0, 0.3258784773328835),
 (('10', '770'), 1, 0.3370607613335583),
 (('10', '770'), 2, 0.3370607613335583),
 (('54', '1097'), 0, 0.15947161581923636),
 (('54', '1097'), 1, 0.4202641920903818),
 (('54', '1097'), 2, 0.4202641920903818),
 (('102', '770'), 0, 0.3258784773328835),
 (('102', '770'), 1, 0.3370607613335583),
 (('102', '770'), 2, 0.3370607613335583),
 (('190', '770'), 0, 0.3258784773328835)]

Update LogLik = sum( log( ∑ p(s | z) * p(z | u))) = sum( log (p(s | u))

In [191]:
LogLik = SumPzuPsz.map(lambda x : x[1]).sum()
LogLik

230.48638694217286

End of the iteration step 